In [10]:
import json
import pickle

def parse_squad(dataset):
    """
    Parses SQUAD database into more readable format. In this case I only care
    about question/answers pairs in order to make a seq2seq model that would
    generate questions out of a paragraph.
    
    Inputs:
        dataset: squad dataset in json format
    Returns:
        squad_json: parsed squad dataset in json format
    """
    total_topics = 0
    total_questions = 0
    squad_json = []
    
    # Iterate through every topic in the dataset
    for topic in dataset:
        total_topics += 1
        # Iterate through every text passage in the topic
        for passage in topic['paragraphs']:     
            # Iterate through every question/answer pairs in the passage
            for qas in passage['qas']:
                total_questions += 1
                text_question_pair = {}
                # Append the title
                text_question_pair['topic'] = topic['title']   
                # Append the text paragraph
                text_question_pair['paragraph'] = passage['context']
                # Append the question
                text_question_pair['question'] = qas['question']
                # Iterate through available answers
                answers = []
                for answer in qas['answers']:
                    answers.append(answer['text'])
                # And append them all together
                text_question_pair['answers'] = answers 
                
                # Append found dictionary to the full parsed dataset array
                squad_json.append(text_question_pair)
    
    print('Found ' + str(total_topics) + ' topics in total.')
    print('Found ' + str(total_questions) + ' questions in total.')
    return squad_json

#==============================================================================
# # PARSE AND SAVE TRAIN DATA
#==============================================================================
squad_train_filepath = '../de.json'
save_path = 'parsed_train_data.json'

# Load squad train dataset
train_json = json.load(open(squad_train_filepath, 'r'))
train_dataset = train_json['data']

parsed_train_squad = parse_squad(train_dataset)
json.dump(parsed_train_squad, open(save_path, 'w'))

#==============================================================================
# # PARSE AND SAVE DEV DATA
#==============================================================================
# Filepath to squad dataset and path where to save the parsed dataset  
squad_dev_filepath = '../de.json'
save_path = 'parsed_dev_data.json'

# Load squad dev dataset
dev_json = json.load(open(squad_dev_filepath, 'r'))
dev_dataset = dev_json['data']

parsed_dev_squad = parse_squad(dev_dataset)
json.dump(parsed_dev_squad, open(save_path, 'w'))

#======================================================================
# Extract paragraph/questions pairs from Stanford's QUAD database
#======================================================================
train_filepath = 'parsed_train_data.json'
dev_filepath = 'parsed_dev_data.json'

train_set = json.load(open(train_filepath, 'r'))
dev_set = json.load(open(dev_filepath, 'r'))

train_paragraphs = []
train_questions = []
for section in train_set:
    train_paragraphs.append(section['paragraph'])
    train_questions.append(section['question'])
    
print train_paragraphs[0]
print train_questions[0]
    
dev_paragraphs = []
dev_questions = []
for section in dev_set:
    dev_paragraphs.append(section['paragraph'])
    dev_questions.append(section['question'])
    
assert len(train_paragraphs) == len(train_questions)
assert len(dev_paragraphs) == len(dev_questions)

#==============================================================================
# # Pickle up the extracted lists of questions/anserws pairs
#==============================================================================
def save_pickle(data, filename):
    """Saves the data into pickle format"""
    save_documents = open(filename+'.pickle', 'wb')
    pickle.dump(data, save_documents)
    save_documents.close()
    
save_pickle(train_paragraphs, 'train_squad_paragraphs')
save_pickle(train_questions, 'train_squad_questions')
save_pickle(dev_paragraphs, 'dev_squad_paragraphs')
save_pickle(dev_questions, 'dev_squad_questions')

Found 35 topics in total.
Found 11873 questions in total.
Found 35 topics in total.
Found 11873 questions in total.
The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
In what country is Normandy located?


In [5]:


import pickle
import spacy
import numpy as np
import pandas as pd

# Load up spacy model and import stop words
nlp = spacy.load('en_core_web_sm')
from spacy.lang.en import STOP_WORDSy
for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True

def save_pickle(data, filename):
    """Saves the data into pickle format"""
    save_documents = open('data/'+filename+'.pickle', 'wb')
    pickle.dump(data, save_documents)
    save_documents.close()
    
def load_pickle(data_filepath):
    """Loads up the pickled dataset for further parsing and preprocessing"""
    documents_f = open('data/'+data_filepath+'.pickle', 'rb')
    data = pickle.load(documents_f)
    documents_f.close()
    
    return data

def preprocess_data(data, remove_stopwords=True, replace_entities=False):
    """
    Preprocesses the data by changing all the entities in text into their
    respective form (PERSON, LOC, GPE etc) as well as removes stopwords and
    punctuations from the text is asked for. For body text stopwords should
    be removed, but questions should stay in the original form as we would 
    want the model to generate proper looking questions.
    
    Inputs:
        data: a list of texts to preprocess
        remove_stopwords: as name suggests, boolean, default True
        replace_entities: change entities into their tags, default False
    Returns:
        parsed_data: a list of input texts, but preprocessed
    """
    parsed_data = []
    for index in range(len(data)):
        # Text being parsed right now
        text = data[index]
        
        if replace_entities:
            # That will take a while for big corpuses
            spacy_text = nlp(text)
            text_ents = [(str(ent), str(ent.label_)) for ent in spacy_text.ents]
            
            text = text.lower()
            # Replace entities
            for ent in text_ents:
                replacee = str(ent[0].lower())
                replacer = str(ent[1])
                try:
                    text = text.replace(replacee, replacer)
                except:
                    pass
        else:
            text = text.lower()
            
        # Deal with stopwords and punctuation
        text = nlp(text)
        if remove_stopwords:
            text = [str(token.orth_) for token in text 
                    if not token.is_stop and not token.is_punct]
            text = ' '.join(text)
        else:
            text = [str(token.orth_) for token in text if not token.is_punct]
            text = ' '.join(text)
            
        parsed_data.append(text)
        
        if index % 100 == 0 and index > 0:
            print('Preprocessing {}/{}'.format(index, len(data)))
            
        if index % 1000 == 0 and index > 0:
            print('Pickling progress so far.')
            save_pickle(parsed_data, 'parsed_data')
         
        if index % 2000 == 0:
            try:
                print('Sanity check, currently parsed text is:')
                print(text)
            except:
                pass

    return parsed_data
    
def load_embeddings(embeddings_index, filepath):
    """Load Numberbatch word embeddings"""
    print('Loading Conceptnet Numberbatch word embeddings')
    with open(filepath, encoding='utf-8') as f:
        for line in f:
            values = line.split(' ')
            word = values[0]
            embedding = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = embedding
    
    print('Word embeddings:', len(embeddings_index))
    
def count_word_frequency(word_frequency, data):
    """calculate the world frequency for each token in the corpus"""
    for text in data:
        for token in text.split():
            if token not in word_frequency:
                word_frequency[token] = 1
            else:
                word_frequency[token] += 1
                
def create_conversion_dictionaries(word_frequency, embeddings_index, threshold=10):
    """
    Cleans the dataset by removing the words from a corpus which appear below
    a predefined threshold (default 20).
    
    Input:
        word_frequency: dictionary with word frequencies in the corpus
        embeddings_index: dictionary of words and their corresponding vectors
        threshold: frequency threshold under which words will be discarded
    Returns:
        vocab2int: dictionary to convert vocabulary to integers
        int2vocab: dictionary to reverse conversion, integers and their vocab
    """
    print('Removing token which frequency in the corpus is under specified threshold')
    missing_words = 0
    
    for token, freq in word_frequency.items():
        if freq > threshold:
            if token not in embeddings_index:
                missing_words += 1
                
    missing_ratio = round(missing_words/len(word_frequency), 4) * 100
    print('Number of words missing from Conceptnet Numberbatch:', missing_words)
    print('Percent of words that are missing from vocabulary: ', missing_ratio, '%')

    # Dictionary to convert words to integers
    print('Creating vocab_to_int dictionary')
    vocab2int = {}
    
    value = 0
    for token, freq in word_frequency.items():
        if freq >= threshold or token in embeddings_index:
            vocab2int[token] = value
            value += 1
    
    # Special tokens that will be added to our vocab. Those tokens will guide the
    # sequence to sequence model
    codes = ['<UNK>', '<PAD>', '<EOS>', '<GO>']   
    
    print('Adding special tokens to vocab_to_int dictionary.')
    # Add the codes to the vocab list
    for code in codes:
        vocab2int[code] = len(vocab2int)
    
    # Dictionary to convert integers to words
    print('Creating int_to_vocab dictionary.')
    int2vocab = {}
    for token, index in vocab2int.items():
        int2vocab[index] = token
    
    usage_ratio = round(len(vocab2int) / len(word_frequency), 4) * 100
    print("Total number of unique words:", len(word_frequency))
    print("Number of words we will use:", len(vocab2int))
    print("Percent of words we will use: {}%".format(usage_ratio))
    
    return vocab2int, int2vocab

def create_embedding_matrix(vocab2int, embeddings_index, embedding_dimensions=300):
    """
    Creates embedding matrix for each token left in the corpus, as denoted by 
    vocab to int. If the word vector is not available in the embeddings_index
    then we create a random embedding for it.
    
    Input:
        vocab2int: dictionary which contains vocab to integer conversions for corpus
        embeddings_index: word vectors
        embedding_dimensions: dimensions of word vectors. 300 by default to match Conceptnet
    Returns:
        word_embedding_matrix: final word vectors for our corpus
    """
    # Number of words in total in the corpus
    num_words = len(vocab2int)
    
    # Create a default matrix with all values set to zero and fill it out
    print('Creating word embedding matrix with all the tokens and their corresponding vectors.')
    word_embedding_matrix = np.zeros((num_words, embedding_dimensions), dtype=np.float32)
    for token, index in vocab2int.items():
        if token in embeddings_index:
            # If the token is pretrained in CN's vectors, use that
            word_embedding_matrix[index] = embeddings_index[token]
        else:
            # Else, create a random embedding for it
            new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dimensions))
            word_embedding_matrix[index] = new_embedding
            
    return word_embedding_matrix
    
def convert_data_to_ints(data, vocab2int, word_count, unk_count, eos=True):
    """
    Converts the words in the data into their corresponding integer values.
    
    Input:
        data: a list of texts in the corpus
        vocab2list: conversion dictionaries
        word_count: an integer to count the words in the dataset
        unk_count: an integer to count the <UNK> tokens in the dataset
        eos: boolean whether to append <EOS> token at the end or not (default true)
    Returns:
        converted_data: a list of corpus texts converted to integers
        word_count: updated word count
        unk_count: updated unk_count
    """    
    converted_data = []
    for text in data:
        converted_text = []
        for token in text.split():
            word_count += 1
            if token in vocab2int:
                # Convert each token in the paragraph to int and append it
                converted_text.append(vocab2int[token])
            else:
                # If it's not in the dictionary, use the int for <UNK> token instead
                converted_text.append(vocab2int['<UNK>'])
                unk_count += 1
        if eos:
            # Append <EOS> token if specified
            converted_text.append(vocab2int['<EOS>'])
            
        converted_data.append(converted_text)
    
    assert len(converted_data) == len(data)
    return converted_data, word_count, unk_count

def build_summary(data):
    """
    Build pandas data frame summary for dataset, useful for finding the length
    our sequence should be
    """
    summary = []
    for text in data:
        summary.append(len(text))
    return pd.DataFrame(summary, columns=['counts'])

def unk_counter(data, vocab2int):
    """Count <UNK> tokens in data"""
    unk_count = 0
    for token in data:
        if token == vocab2int['<UNK>']:
            unk_count += 1
    return unk_count

def remove_wrong_length_data(coverted_inputs, converted_targets, vocab2int,
                             start_inputs_length, max_inputs_length, max_targets_length, 
                             min_inputs_length=10, min_targets_lengths=5,
                             unk_inputs_limit=1, unk_targets_limit=0):
    """
    Sort the paragraphs and questions by the length of their texts, shortest to longest
    Limit the length of summaries and texts based on the min and max ranges.
    This step is important especially if some of the texts in the corpus provided
    are very long, compared to others. For long texts, it would take too much
    memory to learn sequence, thus we want to avoid those. Short sequences
    might act as unneccesary noise in the learning process.
    """
    sorted_inputs = []
    sorted_targets = []
    
    print('Doing final preprocessing - sorting the texts and keeping only those ' + \
          'of appropriate length.')
    for length in range(start_inputs_length, max_inputs_length): 
        for index, words in enumerate(converted_targets):
            if (len(converted_targets[index]) >= min_targets_lengths and
                len(converted_targets[index]) <= max_targets_length and
                len(coverted_inputs[index]) >= min_inputs_length and
                unk_counter(converted_targets[index], vocab2int) <= unk_targets_limit and
                unk_counter(coverted_inputs[index], vocab2int) <= unk_inputs_limit and
                length == len(coverted_inputs[index])
               ):
                sorted_targets.append(converted_targets[index])
                sorted_inputs.append(coverted_inputs[index])
        
    # Ensure the lenght os sorten paragraph and questions match
    assert len(sorted_inputs) == len(sorted_targets)
    print('Got {} inputs/targets pairs!'.format(len(sorted_inputs)))
    
    return sorted_inputs, sorted_targets

#==============================================================================
# # Load the dataset
#==============================================================================            
data_inputs = load_pickle('train_squad_paragraphs')
data_targets = load_pickle('train_squad_questions') 
assert len(data_targets) == len(data_inputs)
print('Loaded {} question/answer pairs.'.format(len(data_inputs)))

#==============================================================================
# # Load up parsed dataset if found. Else preprocess it
#==============================================================================
try:
    parsed_inputs = load_pickle('parsed_inputs')
    parsed_targets = load_pickle('parsed_targets')
except:
    print('Preprocessing inputs, this may take a while...')
    parsed_inputs = preprocess_data(data_inputs, remove_stopwords=True,
                                    replace_entities=True)
    save_pickle(parsed_inputs, 'parsed_inputs')
    print('Preprocessing targets, this may take a while...')
    parsed_targets = preprocess_data(data_targets, remove_stopwords=False,
                                     replace_entities=True)
    save_pickle(parsed_targets, 'parsed_targets')
    
    assert len(parsed_inputs) == len(parsed_targets)
    print('Loaded up {} parsed inputs/targets pairs'.format(len(parsed_inputs)))

pickle_parsed_data = 1
if pickle_parsed_data:
    save_pickle(parsed_inputs, 'parsed_inputs')
    save_pickle(parsed_targets, 'parsed_targets')

#==============================================================================
# # Load Numberbatch word embeddings
#==============================================================================
filepath = '../numberbatch-en-17.06.txt'
embeddings_index = {}
load_embeddings(embeddings_index, filepath)

#==============================================================================
# # Calculate word frequency
#==============================================================================
word_frequency = {}
count_word_frequency(word_frequency, parsed_targets)
count_word_frequency(word_frequency, parsed_inputs)

#==============================================================================
# # Get the usable only tokens and their integer conversion
#==============================================================================
vocab2int, int2vocab = create_conversion_dictionaries(word_frequency, embeddings_index)
save_pickle(vocab2int, 'vocab2int')
save_pickle(int2vocab, 'int2vocab')

#==============================================================================
# Create embedding matrix
#==============================================================================
word_embedding_matrix = create_embedding_matrix(vocab2int, embeddings_index)
del embeddings_index
save_pickle(word_embedding_matrix, 'word_embedding_matrix')

#==============================================================================
# Convert words to integers and pickle the data
#==============================================================================
word_count = 0
unk_count = 0

print('Converting text to integers')
converted_inputs, word_count, unk_count = convert_data_to_ints(parsed_inputs, 
                                                               vocab2int,
                                                               word_count, 
                                                               unk_count)
converted_targets, word_count, unk_count = convert_data_to_ints(parsed_targets, 
                                                                vocab2int,
                                                                word_count, 
                                                                unk_count)
assert len(converted_inputs) == len(converted_targets)

unk_percent = round(unk_count/word_count, 4) * 100
print('Total number of words:', word_count)
print('Total number of UNKs:', unk_count)
print('Percent of words that are UNK:', unk_percent)

save_pickle(converted_inputs, 'converted_inputs')
save_pickle(converted_targets, 'converted_targets')

#==============================================================================
# Build summary and sort the data to keep only the appropriate length
#==============================================================================
assert len(converted_inputs) == len(converted_targets)

summary_inputs = build_summary(converted_inputs)
summary_targets = build_summary(converted_targets)

print('Inputs:')
print(summary_inputs.describe())
print('#' * 50)
print('Targets')
print(summary_targets.describe())

sorted_inputs, sorted_targets = remove_wrong_length_data(converted_inputs,
                                                         converted_targets,
                                                         vocab2int,
                                                         start_inputs_length=min(summary_inputs.counts),
                                                         max_inputs_length=int(np.percentile(summary_inputs.counts, 100)),
                                                         max_targets_length=int(np.percentile(summary_targets.counts, 100)),
                                                         min_inputs_length=10,
                                                         min_targets_lengths=5,
                                                         unk_inputs_limit=1,
                                                         unk_targets_limit=0)

print('Pickling the final files.')
save_pickle(sorted_inputs, 'sorted_inputs')
save_pickle(sorted_targets, 'sorted_targets')

print('Preprocessing data finished!')



IOError: [Errno 2] No such file or directory: '/Users/alakazam/anaconda2/lib/python2.7/site-packages/en_core_web_sm/en_core_web_sm-2.0.0/tagger/model'

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.python.layers.core import Dense
from tensorflow.contrib.seq2seq.python.ops import beam_search_ops

#==============================================================================
# Building the model
#==============================================================================
def model_inputs():
    """Create palceholders for inputs to the model"""
    
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    target_length = tf.placeholder(tf.int32, (None,), name='target_length')
    max_target_length = tf.reduce_max(target_length, name='max_dec_len')
    input_length = tf.placeholder(tf.int32, (None,), name='input_length')

    return input_data, targets, lr, keep_prob, \
           target_length, max_target_length, input_length
           
def process_encoding_input(target_data, vocab2int, batch_size):
    """
    Remove the last word id from each batch and concat the <GO> to the 
    begining of each batch
    """
    
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab2int['<GO>']), ending], 1)

    return dec_input

def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    """Create the encoding layer"""
    
    for layer in range(num_layers):
        with tf.variable_scope('encoder_{}'.format(layer)):
            cell_fw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, 
                                                    input_keep_prob=keep_prob)

            cell_bw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, 
                                                    input_keep_prob=keep_prob)

            enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                                    cell_bw, 
                                                                    rnn_inputs,
                                                                    sequence_length,
                                                                    dtype=tf.float32)
    # Join outputs since we are using a bidirectional RNN
    enc_output = tf.concat(enc_output, 2)
    
    return enc_output, enc_state

def training_decoding_attention(rnn_size, enc_output, enc_state, input_length, 
                                dec_cell, batch_size):
    """Prepare the training attention and initial state"""
    attn_mech_training = tf.contrib.seq2seq.BahdanauAttention(rnn_size,
                                                              enc_output,
                                                              input_length,
                                                              normalize=False,
                                                              name='BahdanauAttention')
    dec_cell_training = tf.contrib.seq2seq.AttentionWrapper(cell=dec_cell,
                                                            attention_mechanism=attn_mech_training,
                                                            attention_layer_size=rnn_size)
    initial_state_training = dec_cell_training.zero_state(batch_size, tf.float32)
    initial_state_training = initial_state_training.clone(cell_state=enc_state[0])
    
    return dec_cell_training, initial_state_training

def training_decoding_layer(dec_embed_input, target_length, dec_cell, initial_state, 
                            output_layer, vocab_size, max_target_length):
    """Create the training logits"""
    
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=target_length,
                                                        time_major=False)
    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       training_helper,
                                                       initial_state,
                                                       output_layer) 
    training_logits = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                           output_time_major=False,
                                                           impute_finished=True,
                                                           maximum_iterations=max_target_length)
    return training_logits[0]

def inference_decoding_attention(enc_output, enc_state, input_length, rnn_size, dec_cell,
                                batch_size, beam_width):
    """Prepare the inferenec attention and initial state"""
    tiled_encoder_outputs = tf.contrib.seq2seq.tile_batch(enc_output, multiplier=beam_width)
    tiled_encoder_final_state = tf.contrib.seq2seq.tile_batch(enc_state[0], multiplier=beam_width)
    tiled_sequence_length = tf.contrib.seq2seq.tile_batch(input_length, multiplier=beam_width)
    
    attn_mech_sample = tf.contrib.seq2seq.BahdanauAttention(num_units=rnn_size,
                                                            memory=tiled_encoder_outputs,
                                                            memory_sequence_length=tiled_sequence_length)
    dec_cell_inference = tf.contrib.seq2seq.AttentionWrapper(cell=dec_cell,
                                                             attention_mechanism=attn_mech_sample,
                                                             attention_layer_size=rnn_size)
    decoder_initial_state_inference = dec_cell_inference.zero_state(dtype=tf.float32, 
                                                                   batch_size=batch_size*beam_width)
    decoder_initial_state_inference = decoder_initial_state_inference.clone(cell_state=tiled_encoder_final_state)
    
    return dec_cell_inference, decoder_initial_state_inference

def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, initial_state, 
                             output_layer, max_target_length, batch_size, beam_width):
    """
    Create the inference logits with beam search, which will be used during
    the sampling stage in order to find a bunch of questions with highest predictions
    """
    
    start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), 
                           [batch_size], 
                           name='start_tokens')    
    inference_decoder = tf.contrib.seq2seq.BeamSearchDecoder(cell=dec_cell,
                                                             embedding=embeddings,
                                                             start_tokens=start_tokens,
                                                             end_token=end_token,
                                                             initial_state=initial_state,
                                                             beam_width=beam_width,
                                                             output_layer=output_layer)
    inference_logits = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                         output_time_major=False,
                                                         impute_finished=False,
                                                         maximum_iterations=max_target_length)
    
    return inference_logits[0]

def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, 
                   input_length, target_length, max_target_length, rnn_size, 
                   vocab2int, keep_prob, batch_size, num_layers, beam_width):
    """
    Create the decoding cell and attention for the training and inference 
    decoding layers. There are different initial state for training and inference
    due to the use of beam_search
    """
    
    for layer in range(num_layers):
        with tf.variable_scope('decoder_{}'.format(layer)):
            lstm = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            dec_cell = tf.contrib.rnn.DropoutWrapper(lstm, 
                                                     input_keep_prob = keep_prob)
    
    output_layer = Dense(vocab_size,
                         kernel_initializer=tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    #==========================================================================
    # # Training decode using standard decoder
    #==========================================================================   
    dec_cell_training, initial_state_training = training_decoding_attention(rnn_size, 
                                                                            enc_output, 
                                                                            enc_state, 
                                                                            input_length, 
                                                                            dec_cell, 
                                                                            batch_size)
    
    with tf.variable_scope("decode"):
        training_logits = training_decoding_layer(dec_embed_input, 
                                                  target_length, 
                                                  dec_cell_training, 
                                                  initial_state_training,
                                                  output_layer,
                                                  vocab_size, 
                                                  max_target_length)
    
    #==========================================================================
    # # Inference decoding using beam search
    #==========================================================================
    dec_cell_inference, decoder_init_state_inference = inference_decoding_attention(enc_output, 
                                                                                    enc_state, 
                                                                                    input_length, 
                                                                                    rnn_size, 
                                                                                    dec_cell,
                                                                                    batch_size, 
                                                                                    beam_width)
        
    with tf.variable_scope("decode", reuse=True):
        inference_logits = inference_decoding_layer(embeddings,  
                                                    vocab2int['<GO>'], 
                                                    vocab2int['<EOS>'],
                                                    dec_cell_inference, 
                                                    decoder_init_state_inference, 
                                                    output_layer,
                                                    max_target_length,
                                                    batch_size,
                                                    beam_width)

    return training_logits, inference_logits

def seq2seq_model(input_data, target_data, keep_prob, input_length, target_length, 
                  max_target_length, vocab_size, rnn_size, num_layers, vocab2int, 
                  word_embedding_matrix, batch_size, beam_width):
    """Use the previously defined functions to create the training and inference logits"""
    
    # Use Numberbatch's embeddings and the newly created ones as our embeddings
    embeddings = word_embedding_matrix
    
    enc_embed_input = tf.nn.embedding_lookup(embeddings, input_data)
    enc_output, enc_state = encoding_layer(rnn_size, input_length, num_layers, 
                                           enc_embed_input, keep_prob)
    
    dec_input = process_encoding_input(target_data, vocab2int, batch_size)
    dec_embed_input = tf.nn.embedding_lookup(embeddings, dec_input)
    
    training_logits, inference_logits = decoding_layer(dec_embed_input, 
                                                       embeddings,
                                                       enc_output,
                                                       enc_state, 
                                                       vocab_size, 
                                                       input_length, 
                                                       target_length, 
                                                       max_target_length,
                                                       rnn_size, 
                                                       vocab2int, 
                                                       keep_prob, 
                                                       batch_size,
                                                       num_layers,
                                                       beam_width)
    
    return training_logits, inference_logits

def pad_text_batch(data_batch, vocab2int):
    """Pad text with <PAD> so that each text of a batch has the same length"""
    max_text = max([len(text) for text in data_batch])
    return [text + [vocab2int['<PAD>']] * (max_text - len(text)) for text in data_batch]

def get_batches(targets, inputs, vocab2int, batch_size):
    """Batch targets, inputs, and the their lengths of their together"""
    for batch_i in range(0, len(inputs)//batch_size):
        start_i = batch_i * batch_size
        targets_batch = targets[start_i:start_i + batch_size]
        inputs_batch = inputs[start_i:start_i + batch_size]
        pad_targets_batch = np.array(pad_text_batch(targets_batch, vocab2int))
        pad_inputs_batch = np.array(pad_text_batch(inputs_batch, vocab2int))
        
        # Need the lengths for the _lengths parameters
        pad_targets_lenghts = []
        for target in pad_targets_batch:
            pad_targets_lenghts.append(len(target))
        
        pad_inputs_lenghts = []
        for text in pad_inputs_batch:
            pad_inputs_lenghts.append(len(text))
        
        yield pad_targets_batch, pad_inputs_batch, pad_targets_lenghts, pad_inputs_lenghts
        